#Data Preprocessing

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Truth_Seeker/cleaned_master_data.csv')
df

,title,text,label,subject,text_length,lemmatized_text
0,\r\r\r\r\r\r\r\nDonald Trump Gets His A Handed...,A 10yearold girl smacked down Donald Trump and...,0,News,2566,yearold girl smacked donald trump amazingon sa...
1,\r\r\r\r\r\r\r\nExGOP Congressman Shreds Fello...,At this point every Republican who refuses to ...,0,News,1925,point every republican refuse spine stand dona...
2,\r\r\r\r\r\r\r\nTrump Gets STOMPED For Whining...,Donald Trump has been attacking President Obam...,0,News,3389,donald trump attacking president obama campaig...
3,\r\r\r\r\r\r\r\nWATCH Democratic Rep Delivers ...,Massachusetts Rep Jim McGovern brought the Hou...,0,News,3358,massachusetts rep jim mcgovern brought house s...
4,AfterTrumpImplodes Hashtag Hilariously Imagin...,What will the world be like postDonald Trump A...,0,News,3092,world like postdonald trump new hashtag aftert...
...,...,...,...,...,...,...
76456,Unknown,you shouldve been swallowed you inbred swine,0,Unknown,44,shouldve swallowed inbred swine
76457,Unknown,Do I know the risk involved in reporting on su...,0,Unknown,127,know risk involved reporting thing reporting t...
76458,Unknown,If conservative Christians are about Jesus Chr...,0,Unknown,2556,conservative christian jesus christ always see...
76459,Unknown,Jesus will save his soul not his body Ignorant...,0,Unknown,60,jesus save soul body ignorant piece shit


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76461 entries, 0 to 76460
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            76460 non-null  object
 1   text             76461 non-null  object
 2   label            76461 non-null  int64 
 3   subject          76461 non-null  object
 4   text_length      76461 non-null  int64 
 5   lemmatized_text  76459 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.5+ MB


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
X = df['lemmatized_text'].astype(str)
y = df['label']

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)

In [ ]:
X = tokenizer.texts_to_sequences(X)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length = 500
truncated_sequences = [seq[:max_sequence_length] for seq in X]
X = pad_sequences(truncated_sequences, maxlen=max_sequence_length)

In [ ]:
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Building RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=X.shape[1]))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.add(SimpleRNN(units=64))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[early_stop])

Epoch 1/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 220s 229ms/step - accuracy: 0.8385 - loss: 0.3633 - val_accuracy: 0.7902 - val_loss: 0.4600
Epoch 2/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 215s 225ms/step - accuracy: 0.8374 - loss: 0.3651 - val_accuracy: 0.7904 - val_loss: 0.4590
Epoch 3/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 218s 228ms/step - accuracy: 0.8386 - loss: 0.3603 - val_accuracy: 0.8173 - val_loss: 0.4208
Epoch 4/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 262s 228ms/step - accuracy: 0.8642 - loss: 0.3170 - val_accuracy: 0.8176 - val_loss: 0.4155
Epoch 5/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 263s 228ms/step - accuracy: 0.8827 - loss: 0.2848 - val_accuracy: 0.8346 - val_loss: 0.4001
Epoch 6/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 256s 222ms/step - accuracy: 0.8937 - loss: 0.2635 - val_accuracy: 0.8333 - val_loss: 0.4029
Epoch 7/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 262s 222ms/step - accuracy: 0.8946 - loss: 0.2624 - val_accuracy: 0.8433 - val_loss: 0.3911
Epoch 8/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 262s 222ms/step - accuracy: 0.9081 -

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

478/478 ━━━━━━━━━━━━━━━━━━━━ 24s 50ms/step - accuracy: 0.8490 - loss: 0.3793
Test Accuracy: 84.99%


In [ ]:
model.save(r"/content/drive/MyDrive/Truth_Seeker/model1.h5")

# LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd

model = Sequential()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Truth_Seeker/cleaned_master_data.csv')
df

,title,text,label,subject,text_length,lemmatized_text
0,\r\r\r\r\r\r\r\nDonald Trump Gets His A Handed...,A 10yearold girl smacked down Donald Trump and...,0,News,2566,yearold girl smacked donald trump amazingon sa...
1,\r\r\r\r\r\r\r\nExGOP Congressman Shreds Fello...,At this point every Republican who refuses to ...,0,News,1925,point every republican refuse spine stand dona...
2,\r\r\r\r\r\r\r\nTrump Gets STOMPED For Whining...,Donald Trump has been attacking President Obam...,0,News,3389,donald trump attacking president obama campaig...
3,\r\r\r\r\r\r\r\nWATCH Democratic Rep Delivers ...,Massachusetts Rep Jim McGovern brought the Hou...,0,News,3358,massachusetts rep jim mcgovern brought house s...
4,AfterTrumpImplodes Hashtag Hilariously Imagin...,What will the world be like postDonald Trump A...,0,News,3092,world like postdonald trump new hashtag aftert...
...,...,...,...,...,...,...
76456,Unknown,you shouldve been swallowed you inbred swine,0,Unknown,44,shouldve swallowed inbred swine
76457,Unknown,Do I know the risk involved in reporting on su...,0,Unknown,127,know risk involved reporting thing reporting t...
76458,Unknown,If conservative Christians are about Jesus Chr...,0,Unknown,2556,conservative christian jesus christ always see...
76459,Unknown,Jesus will save his soul not his body Ignorant...,0,Unknown,60,jesus save soul body ignorant piece shit


In [ ]:
X = df['lemmatized_text'].astype(str)
y = df['label']

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

In [ ]:
import numpy as np
y = np.array(y)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length = 500
truncated_sequences = [seq[:max_sequence_length] for seq in X]
X = pad_sequences(truncated_sequences, maxlen=max_sequence_length)

In [ ]:
model.add(Embedding(input_dim=10000, output_dim=128, input_length=X.shape[1]))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.add(LSTM(units=64, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.3))

In [ ]:
model.add(LSTM(units=32))
model.add(BatchNormalization())
model.add(Dropout(0.3))

In [ ]:
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[early_stop])

Epoch 1/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 1101s 1s/step - accuracy: 0.8171 - loss: 0.4090 - val_accuracy: 0.8990 - val_loss: 0.2406
Epoch 2/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 1104s 1s/step - accuracy: 0.9160 - loss: 0.2088 - val_accuracy: 0.9210 - val_loss: 0.1957
Epoch 3/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 1110s 1s/step - accuracy: 0.9434 - loss: 0.1503 - val_accuracy: 0.9113 - val_loss: 0.2181
Epoch 4/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 1123s 1s/step - accuracy: 0.9629 - loss: 0.1050 - val_accuracy: 0.9176 - val_loss: 0.2242
Epoch 5/10
956/956 ━━━━━━━━━━━━━━━━━━━━ 1160s 1s/step - accuracy: 0.9714 - loss: 0.0805 - val_accuracy: 0.9187 - val_loss: 0.2681


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

478/478 ━━━━━━━━━━━━━━━━━━━━ 71s 149ms/step - accuracy: 0.9201 - loss: 0.1988
Test Accuracy: 92.10%


In [ ]:
model.save(r"/content/drive/MyDrive/Truth_Seeker/lstm_model.h5")